Course: Computational Statistics | SS2020 | University of Bonn
# Topic: Comparison in algorithms for Classification Tree

Author: Po-Tse Cheng

maybe add some structure here? (see if we got enough space)

In [1]:
#install packages in need
install.packages(c("partykit", "RWeka", "rpart", "truncnorm", "remotes","wakefield", "partykit", "data.table"))

#initial the packages require for the projects:
library(wakefield)
library(partykit)
library(truncnorm)
library(rpart)
library(RWeka)
library(partykit)
library(data.table)

#import the custom DGP function from R codes in the repository
source("R codes/DGP.R")

#supress unwanted warning in loading packages
#options(warn=-1)
#do this in the end

Installing packages into 'C:/Users/Viktor Cheng/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'partykit' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'partykit'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"複製 C:\Users\Viktor Cheng\Documents\R\win-library\3.6\00LOCK\partykit\libs\x64\partykit.dll 到 C:\Users\Viktor Cheng\Documents\R\win-library\3.6\partykit\libs\x64\partykit.dll 時出了問題：Permission denied"Warning message:
"restored 'partykit'"

package 'RWeka' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'RWeka'"Warning message:
"restored 'RWeka'"

package 'rpart' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rpart'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"複製 C:\Users\Viktor Cheng\Documents\R\win-library\3.6\00LOCK\rpart\libs\x64\rpart.dll 到 C:\Users\Viktor Cheng\Documents\R\win-library\3.6\rpart\libs\x64\rpart.dll 時出了問題：Permission denied"Warning message:
"restored 'rpart'"

package 'truncnorm' successfully unpacked and MD5 sums checked
package 'remotes' successfully unpacked and MD5 sums checked
package 'wakefield' successfully unpacked and MD5 sums checked
package 'data.table' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Viktor Cheng\AppData\Local\Temp\RtmpSAQ5pF\downloaded_packages


Warning message:
"package 'wakefield' was built under R version 3.6.3"Warning message:
"package 'partykit' was built under R version 3.6.3"Loading required package: grid
Loading required package: libcoin
Warning message:
"package 'libcoin' was built under R version 3.6.3"Loading required package: mvtnorm
Warning message:
"package 'mvtnorm' was built under R version 3.6.3"Warning message:
"package 'truncnorm' was built under R version 3.6.3"Warning message:
"package 'rpart' was built under R version 3.6.3"Warning message:
"package 'RWeka' was built under R version 3.6.3"Warning message:
"package 'data.table' was built under R version 3.6.3"
Attaching package: 'data.table'

The following objects are masked from 'package:wakefield':

    hour, minute, month, second, year



## 1. Introduction

The main purpose of this notebook is to showcase the strengh and weekness of different algorithms in classification tree, in particular, the decision making mechanism behind the splits between nodes. The motivation behind this attempt is to understand each individual splitting methods more thoroughly, as within the courses


Two seperate DGPs with different characteristics will be utilised to help evaulate the algorithms. In total 3( if got time 4) will be discussed:
<ul>
    <li>Classification and Regression Trees (CART)</li>
    <li>C4.5</li>
    <li>Conditional Inference Trees (Ctree)</li>
</ul>

The most direct algorithm for determining a split is the <em>Gini Index</em>, which is employed by CART. C4.5 uses the <em>Shannon Entropy </em>method which in some sense is simliar to Gini Index. Lastly, Ctree incoporates the <em>Chi-square test</em> to determine the split. In this notebook,  we will be using the rpart package for CART algorithm, rweka for J48, and finally partykit package for Ctree. This notebook is also constructed in the following way: in the second chapter we will take a look at the two DGPs including personal assumptions and the mechanism of each algorithms. Section 3 is using the examples 



<div class="alert alert-block alert-info">
    <b>Additional Note:</b> The DGP functions are quite long, therefore I decided to import it from the repository. For the sake of transparency, please <a href="https://github.com/PoTseCheng/Computational-Statistic/blob/master/R%20codes/DGP.R">click here</a> to evaluate the code. 
    
</div>

## 2. DGPs and Theoretical Perspectives

### 2.1. Data Generating Procedures

In this section we will be going through the two main DGPs: The first one came from Tian et al.(2020) from Journal of Infection, while the second one is from Jones-White et al.(2014) published in Research in Higher Education. The first paper analysed the symptoms of COVID-19 and the consequences of all 262 cases. Furthermore, the 262 cases were splitted into 2 subgroups by clinical perspective. The other paper aims to establish a connection between financial aids and students' college outcomes. 


Below is a short summary of data from the original papers:

<table><tr>
<thead>
 <tr><th>Description Statistics 1</th><th>Description Statistics 2</th></tr>   
</thead>
    
<tbody>
    
 <tr><td> <img src="Pics/DGP1.png"> </td><td> <img src="Pics/DGP2.png"> </td></tr>

</tbody>
</table>

These two papers were intentially selected as there are a huge contrast in complexity 

Below are a portion of samples from our DGP. Showcase1 denotes

In [2]:
set.seed(123)
data1<- Observations(100)
data2<- Observations2(100)

In [3]:
showcase1<- data.table(data1)
showcase1[sample(.N, 6)]

severity,gender,age,address,contact,temperature,tlabel,symptoms,result
0,Female,42,Beijin,Wuhan 14 days,37.62,Fever,No Symptoms,Hospitalization
0,Female,32,Wuhan,Wuhan 14 days,37.61,Fever,Headache,Hospitalization
0,Female,93,Beijin,Wuhan 14 days,38.51,Fever,Coughing,Hospitalization
0,Male,37,Beijin,Wuhan 14 days,37.78,Fever,No Symptoms,Hospitalization
1,Female,83,Beijin,Case 14 days,38.74,Fever,"Coughing, Dyspnea",Hospitalization
1,Female,89,Beijin,Unknown,38.81,Fever,Coughing,Hospitalization


In [4]:
showcase2<- data.table(data2)
showcase2[sample(.N, 6)]

Gender,Race,Generation,Status,Geographic,Remedy,Choice,Living,Community,Athlete,Aidtype,Aidamount,Unmet,ACT,AP,Course,Ccount,Dcount,result
Female,White,No,Traditional,In-state,No,First Choice,Outside Campus,No,No,Loan Aid,4.71,6.41,30.96,0.40,57.53,0.98,0.13,Graduated
Male,White,Yes,Traditional,In-state,No,Others,On Campus,No,No,Loan Aid,4.36,8.66,33.06,7.03,92.95,1.59,0.45,Graduated
Female,White,No,Traditional,In-state,Yes,Others,Outside Campus,Yes,No,Need Aid,4.72,2.29,23.63,14.33,61.46,1.26,0.49,Not Graduated
Female,White,No,Traditional,Reciprocity state,No,First Choice,Outside Campus,No,No,Loan Aid,1.56,8.13,25.54,5.45,67.01,0.16,0.03,Graduated
Female,White,No,Traditional,In-state,No,First Choice,Outside Campus,Yes,No,Need Aid,0.55,0.25,22.23,7.09,97.94,1.09,0.18,Graduated
Male,White,No,Traditional,Reciprocity state,No,Others,On Campus,No,No,Merit Aid,0.40,0.50,28.12,13.21,97.38,0.47,0.36,Graduated


### 2.2. Mechanisms and Expectations

We expect that

## Algorithm Evaluation

In this cha

In [ ]:
algorithm: CART

In [ ]:
SOME  OTHER SHIT REGARDING C4.5

In [ ]:
algorithm:c4.5 with J48

In [ ]:
CTree

Benchmark accuracy/ also give ratings

additional chapter (Cruise):
addition since the package is not avaliable in r, one must download the algorithm manually from the ... website.

Reference:

Jones-White, D. R., Radcliffe, P. M., Lorenz, L. M., & Soria, K. M. (2014). Priced out?. Research in Higher Education, 55(4), 329-350.

Loh, W. Y. (2014). Classification and regression tree methods. Wiley StatsRef: Statistics Reference Online.

Loh, W. Y., & Shih, Y. S. (1997). Split selection methods for classification trees. Statistica sinica, 815-840.

Tian, S., Hu, N., Lou, J., Chen, K., Kang, X., Xiang, Z., ... & Chen, G. (2020). Characteristics of COVID-19 infection in Beijing. Journal of Infection.